In [1]:
%cd D:/datasets/futureMAG

D:\datasets\futureMAG


In [2]:
%pwd

'D:\\datasets\\futureMAG'

In [1]:
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import requests

import csv
import os
import sys
import re

### Download montag.wtf

In [4]:
articles = []
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
def get_links(i):
    with requests.get(f'https://www.montag.wtf/page/{i}/', headers=headers, timeout=10) as f:
        soup = BeautifulSoup(f.text, features="html.parser")
        for article in soup.findAll(class_="homepage-post-info"):
            for x in article.findAll("a"):
                articles.append(f'https://www.montag.wtf{x["href"]}')

In [5]:
%%time
with ThreadPoolExecutor(27) as executor:
    results = executor.map(get_links, range(1,27))

Wall time: 1.05 s


In [6]:
len(articles)

156

In [8]:
def parse(url):
    with requests.get(url, headers=headers) as f:
        soup = BeautifulSoup(f.text, features='html.parser')
    article_text = soup.findAll(class_='content-text')
    words = "<|startoftext|>"
    for element in article_text:
        words += element.get_text()
    with open('montag.txt', 'a', encoding='utf-8') as f:
        f.write(words + "<|endoftext|>\n")

In [9]:
%%time

with ThreadPoolExecutor(100) as executor:
    results = executor.map(parse, articles[1:])

Wall time: 1.63 s


### Download OneZero

In [8]:
from selenium import webdriver
import shutil
import time

In [18]:
url = 'https://onezero.medium.com/trending'
driver = webdriver.Chrome()
driver.get(url)

for _ in range(1000):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
#     time.sleep(.5)

articles = []
for date in driver.find_elements_by_tag_name('time'):
    link = date.find_element_by_xpath('..').get_attribute('href')
    articles.append(link)

driver.quit()

In [19]:
len(articles)

199

In [20]:
for i in range(0, 10):
    articles[i] = re.findall('(.*)\?', articles[i])[0]
for i in range(len(articles)):
    articles[i] += '?format=json'

In [29]:
with open('onezero_urls.txt', 'a') as f:
    for link in set(articles):
        f.write(link)

In [22]:
articles[-1]

'https://onezero.medium.com/online-conspiracy-theories-start-with-the-young-5774f221a436?format=json'

In [23]:
with open('onezero_urls.txt', 'r') as f:
    articles = f.readlines()

519

In [21]:
# https://hackersandslackers.com/extract-data-from-complex-json-python/
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [30]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
def parse(url):
    with requests.get(url, headers=headers) as response:
        soup = BeautifulSoup(response.text, features='html.parser')
#     article_text = soup.find(class_='section-inner')
#     with open("onezero.txt", "a") as f:
#         f.write('\n' + article_text.find('h1').get_text() + '\n')
#         f.write(article_text.find('h2').get_text() + '\n')
#         for element in article_text.findAll('p'):
#             words = element.get_text() + '\n'
#             f.write(words)
    image = soup.find(class_='progressiveMedia-image')
    image_url = image['data-src']
    response = requests.get(image_url, stream=True)
    image_name = re.findall('\*(.*)', image_url)[0]
    with open(f'D:/datasets/fake_news/hero_images/one_zero/{image_name}', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

In [22]:
def parse_json(url):
    with requests.get(url) as response:
        article = json.loads(response.text[16:])
    text = extract_values(article, 'text')
    words = "<|startoftext|>"
    for paragraph in text:
            words += paragraph + '\n'

    with open('onezero.txt', 'a', encoding='utf-8') as f:
        f.write(words + "<|endoftext|>\n")

In [23]:
%%time

with ThreadPoolExecutor(100) as executor:
    results = executor.map(parse_json, articles)

Wall time: 7.98 s


### Download Futura

In [6]:
from selenium import webdriver
import shutil
import time
import json
import re

In [3]:
url = 'https://medium.com/short-sci-fi-stories/latest'
driver = webdriver.Chrome()
driver.get(url)

for _ in range(50):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    time.sleep(.5)

articles = []
for date in driver.find_elements_by_tag_name('time'):
    link = date.find_element_by_xpath('..').get_attribute('href')
    articles.append(link)

driver.quit()

In [4]:
len(articles)

86

In [7]:
for i in range(0, 10):
    articles[i] = re.findall('(.*)\?', articles[i])[0]
for i in range(len(articles)):
    articles[i] += '?format=json'

In [11]:
with open('futura_urls.txt', 'a') as f:
    for link in set(articles):
        f.write(link + '\n')

In [28]:
# https://hackersandslackers.com/extract-data-from-complex-json-python/
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [30]:
def parse_json(url):
    with requests.get(url) as response:
        article = json.loads(response.text[16:])
    text = extract_values(article, 'text')
    words = "<|startoftext|>"
    for paragraph in text:
            words += paragraph + '\n'

    with open('futura.txt', 'a', encoding='utf-8') as f:
        f.write(words + "<|endoftext|>\n")

In [31]:
%%time

with ThreadPoolExecutor(80) as executor:
    results = executor.map(parse_json, articles)

Wall time: 1.34 s
